# Extracción de características básicas de texto
Vamos a crear un generador para procesar un lote de archivos de texto y extraer ciertas características básicas obtenidas con la librería `spaCy`sobre el texto.


In [10]:
import spacy
import pandas as pd
import numpy as np

nlp = spacy.load("es_core_news_sm")

## Carga de datos
Creamos una función generadora para cargar los documentos de texto desde una carpeta de disco, y vamos a realizar un procesado inicial a cada documento.  
Usaremos un conjunto de noticias deportivas del Marca, obtenido de https://www.kaggle.com/datasets/mdamsterdam/marca-spanish-sports-news

In [11]:
import os

def carga_textos(path):
    """Función generadora que carga los archivos de tipo TXT de una carpeta
    Devuelve (yield) el siguiente texto en cada ejecución"""

    for file in [f for f in os.listdir(path) if f.endswith('.txt')]:
        with open(os.path.join(path, file), encoding='utf-8') as f:
            
            yield f.read()

In [12]:
textos = carga_textos("noticias")
print(next(textos))

Una Copa blindada para ganar al covid-19 en Madrid Dubljevic: "No somos favoritos en la Copa porque jugamos primero con el Madrid" El base brasileño del Lenovo Tenerife Marcelinho Huertas disputará su segunda Copa del Rey como aurinegro y su décima como jugador de la Liga Endesa, un torneo que "siempre es especial", según ha declarado en un audio enviado por el club. "Llegamos con mucha motivación, no puede ser de otra manera, una vez más estamos ahí, y hemos llegado como cabezas de serie", ha destacado el jugador que ha insistido en que el único rival en el que piensa el combinado canarista es en el Hereda San Pablo Burgos. Los tinerfeños y el combinado burgalés, recién proclamado campeón de la Copa Intercontinental de la FIBA en Buenos Aires, se medirán en el encuentro inaugural de la cita copera este jueves 11 de febrero a las 18.30 CET (17.30 GMT), en el Wizink Center de Madrid. Huertas ha indicado que el equipo ha mejorado a lo largo de la temporada, y que ha tenido "una dinámica 

In [13]:
print(next(textos))

La Audiencia Provincial de Palma ha decidido reabrir la causa por el fallecimiento de Ángel Nieto, ocurrido el pasado 3 de agosto de 2017 y ha tomado en cuenta el recurso de los hijos mayores del piloto, Pablo y Gelete, considerando que hay indicios suficientes de homicidio por imprudencia. En un primer momento el juzgado de Ibiza, donde se produjo el accidente. cerró el caso en diciembre del mismo año al considerar que la ciudadana alemana que conducía el Fiat 600 que golpeó el quad del 12+1 veces campeón del mundo no cometió ninguna infracción penal. Además Nieto habría llevado el casco sin atar del todo y las luces del vehículo tapadas por el barro, según el parte policial. Incluso su viuda aceptó un acuerdo de 900.000 euros de la aseguradora del vehículo. Sin embargo parte de los hijos del legendario piloto siguieron adelante entendiendo que había más evidencias y ahora la ciudadana alemana, de 41 años, tendrá que declarar en calidad de investigada por ese presunto delito de homici

### Extracción de características básicas
Creamos una función que a partir de un texto de entrada (objeto `string`) genere un diccionario de características con los siguientes valores:  
- `caracteres`: longitud del texto en caracteres
- `palabras`: nº de palabras del texto excluyendo tokens de puntuación
- `frases`: nº de frases del texto
- `ENT_PER`: nº de entidades de tipo `PER` en el texto
- `ENT_LOC`: nº de entidades de tipo `LOC` en el texto
- `ENT_ORG`: nº de entidades de tipo `ORG` en el texto
- `ENT_MISC`: nº de entidades de tipo `MISC` en el texto  

In [14]:
from collections import Counter


def caracteristicas(texto):
    """Calcula una serie de características de un texto
    y las devuelve como valores de un objeto diccionario"""

    diccionario = dict()
    diccionario["caracteres"] = len(texto)
    doc = nlp(texto)
    diccionario["palabras"] = len([t for t in doc if not t.is_punct])
    diccionario["frases"] = len([s for s in doc.sents])
    entidades = Counter([e.label_ for e in doc.ents])
    diccionario["ENT_PER"] = entidades["PER"]
    diccionario["ENT_LOC"] = entidades["LOC"]
    diccionario["ENT_ORG"] = entidades["ORG"]
    diccionario["ENT_MISC"] = entidades["MISC"]

    return diccionario


In [15]:
caracteristicas(next(textos))

{'caracteres': 1260,
 'palabras': 213,
 'frases': 9,
 'ENT_PER': 7,
 'ENT_LOC': 3,
 'ENT_ORG': 1,
 'ENT_MISC': 7}

La aplicaremos sobre el generador anterior para crear un DataFrame con los datos de todo el dataset de noticias.


In [16]:
caract_noticias = pd.DataFrame([caracteristicas(texto) for texto in carga_textos("noticias")])

In [17]:
caract_noticias.sample(10)

,caracteres,palabras,frases,ENT_PER,ENT_LOC,ENT_ORG,ENT_MISC
547,6516,1067,31,26,49,5,137
84,1409,243,10,9,3,2,7
440,2033,341,12,6,8,0,8
70,1843,261,11,9,28,2,17
130,981,179,4,1,4,2,7
430,1777,300,7,8,4,2,11
456,903,162,4,5,4,1,6
386,1254,209,7,8,6,2,9
156,1828,314,9,6,7,3,15
221,3212,575,26,28,7,2,16


Vemos estadísticos globales

In [18]:
caract_noticias.describe()

,caracteres,palabras,frases,ENT_PER,ENT_LOC,ENT_ORG,ENT_MISC
count,561.000000,561.000000,561.000000,561.000000,561.000000,561.000000,561.000000
mean,2429.572193,420.436720,19.146168,14.775401,10.718360,4.427807,14.641711
std,1652.645616,287.181351,20.291713,15.695979,10.192741,4.537803,14.956901
min,124.000000,23.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,1429.000000,249.000000,8.000000,6.000000,4.000000,2.000000,7.000000
50%,2072.000000,357.000000,14.000000,11.000000,8.000000,3.000000,11.000000
75%,3017.000000,511.000000,21.000000,18.000000,14.000000,6.000000,16.000000
max,16002.000000,2712.000000,196.000000,180.000000,66.000000,41.000000,137.000000


la librería `textaCy` tiene muchas funciones para extraer información básica de un texto: https://textacy.readthedocs.io/en/latest/api_reference/text_stats.html